## Instagram crawler sample

In [ ]:
from selenium import webdriver
import urllib.parse
from urllib.request import Request, urlopen
import time
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
chromedriver="./chromedriver_win32/chromedriver.exe"
driver=webdriver.Chrome(chromedriver)

In [ ]:
# login
loginUrl = 'https://www.instagram.com/accounts/login/'
driver.get(loginUrl)
time.sleep(2)

username = input("유저 아이디 입력: ")
userpw="Yunastellar"

driver.find_element_by_name('username').send_keys(username)
driver.find_element_by_name('password').send_keys(userpw)
time.sleep(2)
driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF').click()
time.sleep(3)

In [ ]:
# 정보 나중에 저장하기 클릭하고 넘어가기
try:
    driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
    time.sleep(3)
except:
    pass
# 설정 나중에하기 클릭하고 넘어가기
try:
    driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
    time.sleep(3)
except:
    pass

In [ ]:
keyword=input("해시태그를 입력하세요: ")
encText = urllib.parse.quote(keyword)

In [ ]:
url="https://www.instagram.com/explore/tags/"+str(encText)+'/'
driver.get(url)
time.sleep(3)

In [ ]:
driver.find_element_by_css_selector('div.v1Nh3.kIKUG._bz0w').click()
time.sleep(3)